## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [8]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [9]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros')
        self.dense2 = layers.Dense(10, kernel_initializer='glorot_uniform', bias_initializer='zeros')

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = self.flatten(x)
        x = self.dense1(x)
        logits = self.dense2(x)
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [10]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.dense1.kernel, model.dense2.kernel, model.dense1.bias, model.dense2.bias]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [11]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.4826145 ; accuracy 0.087233335
epoch 1 : loss 2.4655108 ; accuracy 0.09066667
epoch 2 : loss 2.4491735 ; accuracy 0.09395
epoch 3 : loss 2.4335172 ; accuracy 0.09736667
epoch 4 : loss 2.418483 ; accuracy 0.100933336
epoch 5 : loss 2.4040098 ; accuracy 0.10455
epoch 6 : loss 2.3900447 ; accuracy 0.1081
epoch 7 : loss 2.3765428 ; accuracy 0.112166665
epoch 8 : loss 2.3634572 ; accuracy 0.1159
epoch 9 : loss 2.3507593 ; accuracy 0.119883336
epoch 10 : loss 2.3384156 ; accuracy 0.12443333
epoch 11 : loss 2.3263955 ; accuracy 0.12866667
epoch 12 : loss 2.314673 ; accuracy 0.13298333
epoch 13 : loss 2.3032255 ; accuracy 0.13763334
epoch 14 : loss 2.2920299 ; accuracy 0.14238334
epoch 15 : loss 2.281072 ; accuracy 0.1469
epoch 16 : loss 2.2703352 ; accuracy 0.15191667
epoch 17 : loss 2.2598011 ; accuracy 0.15695
epoch 18 : loss 2.2494562 ; accuracy 0.16211666
epoch 19 : loss 2.239286 ; accuracy 0.16826667
epoch 20 : loss 2.2292802 ; accuracy 0.17433333
epoch 21 : loss 2.21943